In [101]:
from ROOT import *
import numpy as np
import math
import time
import matplotlib.pyplot as plt

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

#gROOT.ProcessLine(".x ~/lhcbStyle.C")
#gStyle.SetPaintTextFormat("1.3f")



In [3]:
#OLD FILE UT CONFIG

ch1=TChain("MCParticleNTuple/Tracks")
ch1.Add("~/MightyIT/MCtracks_oldfile_20ev.root")
ch2=TChain("MCParticleNTuple/Tracks")
ch2.Add("~/MightyIT/MCtracks_oldfile_20ev.root")

#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_oldfile_nocuts_20ev.root")
#ch2=TChain("MCParticleNTuple/Tracks")
#ch2.Add("~/MightyIT/MCtracks_oldfile_nocuts_20ev.root")

zmag = 5150. #for MCtracks_oldfile WITH UT


#no smearing sigmas for oldfile
sigmax_nosm=6.3
sigmay_nosm=3.9
sigmatx_nosm=0.0022
sigmaty_nosm=0.0009

sigmax_wsm=55
sigmay_wsm=4
sigmatx_wsm=0.02
sigmaty_wsm=0.06



perc_list = [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
thr_nosm_list = [0.96, 1.08, 1.21, 1.32, 1.44, 1.61, 1.79, 1.98, 2.28, 2.65, 3.14, 3.74, 4.62, 5.57, 7.03, 8.93, 11.58, 15.11, 20.76, 40.05]
thr_wsm_list = [3.05, 3.33, 3.64, 3.98, 4.32, 4.76, 5.29, 5.94, 6.66, 7.5, 8.8, 10.19, 12.14, 15.2, 18.29, 22.06, 28.85, 40.16, 60.07, 95.15]



In [102]:
# NEW FILE UT CONFIG

ch1=TChain("MCParticleNTuple/Tracks")
ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_20ev.root")
ch2=TChain("MCParticleNTuple/Tracks")
ch2.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_20ev.root")

#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_nocuts_20ev.root")
#ch2=TChain("MCParticleNTuple/Tracks")
#ch2.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_nocuts_20ev.root")


zmag = 5144.5 #for MCtracks_MagUp_bs2phiphi_1p5e34 WITH UT

sigmax_nosm=9.2
sigmay_nosm=4.4
sigmatx_nosm=0.0034
sigmaty_nosm=0.0012

sigmax_wsm=24
sigmay_wsm=4.5
sigmatx_wsm=0.009
sigmaty_wsm=0.02

perc_list = [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
thr_nosm_list = [1.15, 1.24, 1.34, 1.52, 1.69, 1.89, 2.07, 2.31, 2.61, 3.03, 3.55, 4.18, 4.86, 6.18, 7.72, 10.28, 14.67, 21.02, 33.97, 56.34]
thr_wsm_list = [2.42, 2.63, 2.85, 3.09, 3.42, 3.73, 4.1, 4.54, 5.05, 5.69, 6.63, 7.82, 8.94, 11.23, 13.73, 17.84, 23.09, 32.18, 44.13, 68.5]
       
zmag = 5106.5 #for MCtracks_MagUp_bs2phiphi_1p5e34

sigmax_nosm=11.6
sigmay_nosm=8.1
sigmatx_nosm=0.0042
sigmaty_nosm=0.0020

sigmax_wsm=28
sigmay_wsm=8.1
sigmatx_wsm=0.01
sigmaty_wsm=0.02

perc_list = [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
thr_nosm_list = [1.41, 1.58, 1.69, 1.87, 2.03, 2.21, 2.45, 2.71, 3.03, 3.39, 3.92, 4.5, 5.28, 6.37, 7.79, 9.98, 14.5, 22.18, 36.23, 77.76]
thr_wsm_list = [3.03, 3.25, 3.46, 3.74, 4.06, 4.39, 4.74, 5.25, 5.86, 6.51, 7.35, 8.42, 9.63, 11.6, 14.31, 17.56, 23.2, 34.31, 51.55, 91.08]
       









In [103]:
# pixel smearing in mightyIT
resx_mit = 0.1/TMath.Sqrt(12)
resy_mit = 0.4/TMath.Sqrt(12)
# strips smearing outside mightyIT
resx_sf = 0.25/TMath.Sqrt(12)
resy_sf = 0.25/TMath.Cos(5.*np.pi/180.) #takes into account 5° tilting


In [104]:
#function that checks if a point (format: (x,y)) is inside the MightyIT region

def MightyITregion():
    dx = 540
    dy = 200
    
    polygon = Polygon([(-4*dx,1*dy),(-3*dx,1*dy),(-3*dx,2*dy),(-2*dx,2*dy),(-2*dx,3*dy),(-1*dx,3*dy),(-1*dx,4*dy),
                      (1*dx,4*dy),(1*dx,3*dy),(2*dx,3*dy),(2*dx,2*dy),(3*dx,2*dy),(3*dx,1*dy),(4*dx,1*dy),
                      (4*dx,-1*dy),(3*dx,-1*dy),(3*dx,-2*dy),(2*dx,-2*dy),(2*dx,-3*dy),(1*dx,-3*dy),(1*dx,-4*dy),
                      (-1*dx,-4*dy),(-1*dx,-3*dy),(-2*dx,-3*dy),(-2*dx,-2*dy),(-3*dx,-2*dy),(-3*dx,-1*dy),(-4*dx,-1*dy)])
    return polygon
    
def is_inregion(point):
    point = Point(point[0], point[1])
           
    region = MightyITregion()
    if region.contains(point):
        return True
    else:
        return False
    

In [105]:
#function which calculates the chi2, also checks if t track is in window if flag is set True

def Slopexy(x0, y0, z0, x1, y1, z1):
    slopex=(x1 - x0)/(z1 - z0)
    slopey=(y1 - y0)/(z1 - z0)
    return slopex,slopey

In [106]:
smearing = False
percentage = 97

##### INITIALISE VARIABLES #####

timestart = time.time()

nVeloUTTracks = 0
nentries = ch1.GetEntries()

index_perc = perc_list.index(percentage)
threshold_chi2 = thr_nosm_list[index_perc]

#lists for slopes
txVeloVelo_list=[]
tyVeloVelo_list=[]
txVeloUT_list=[]
tyVeloUT_list=[]
txUTUT_list=[]
tyUTUT_list=[]

difftx_vv_uu_list = []
diffty_vv_uu_list = []

p_list=[]
p_list_10 = []

print '\n\n\n\npercentage: ', percentage 
print 'smearing: ', smearing 
print 'threshold chi2: : ', threshold_chi2 

##### LOOP #####

for event in ch1:
    
    #positive charge
    if event.qop > 0:
        
        #loop over all tracks with two hits in velo and two hits in UT (not in T stations)
        if ((0.<event.HitVeloZpos[0]<event.HitVeloZpos[1]<800.) and (event.HitUTZpos[1]>event.HitUTZpos[0]>2000) and (event.HitUTZpos[0]<event.HitUTZpos[1]<3000) and event.p>5000
           and (event.HitUTZpos[0]-event.HitVeloZpos[0] != 0) and (event.HitVeloZpos[1]-event.HitVeloZpos[0] != 0) and (event.HitUTZpos[1]-event.HitUTZpos[0] != 0) ):
            nVeloUTTracks +=1
            
            
            if event.p < 10000:
                p_list_10.append(event.p)
                

            #save fist two hits in velo and first two in UT
            xv0=event.HitVeloXpos[0];  yv0 = event.HitVeloYpos[0]; zv0 = event.HitVeloZpos[0];
            xv1=event.HitVeloXpos[1];  yv1 = event.HitVeloYpos[1]; zv1 = event.HitVeloZpos[1];
            xu0=event.HitUTXpos[0];  yu0 = event.HitUTYpos[0]; zu0 = event.HitUTZpos[0];
            xu1=event.HitUTXpos[1];  yu1 = event.HitUTYpos[1]; zu1 = event.HitUTZpos[1];

            slopevv = Slopexy(xv0,yv0,zv0,xv1,yv1,zv1)
            slopevu = Slopexy(xv0,yv0,zv0,xu0,yu0,zu0)
            slopeuu = Slopexy(xu0,yu0,zu0,xu1,yu1,zu1)

            txVeloVelo_list.append(slopevv[0])
            tyVeloVelo_list.append(slopevv[1])
            txVeloUT_list.append(slopevu[0])
            tyVeloUT_list.append(slopevu[1])
            txUTUT_list.append(slopeuu[0])
            tyUTUT_list.append(slopeuu[1])

            difftx_vv_uu_list.append(slopevv[0]-slopeuu[0])
            diffty_vv_uu_list.append(slopevv[1]-slopeuu[1])

            p_list.append(event.p)
        
    if nVeloUTTracks > 10000:
        break
        
    
        
        







percentage:  97
smearing:  False
threshold chi2: :  22.18


In [107]:
Nbins=1000

print(len(p_list))
print(len(p_list_10))

difftx_vvuu=TH2F("difftx_vvuu", "", Nbins,5000,50000,Nbins,-0.05,0.05)
diffty_vvuu=TH2F("diffty_vvuu", "", Nbins,5000,50000,Nbins,-0.05,0.05)

reldifftx_vvuu=TH2F("reldifftx_vvuu", "", Nbins,5000,50000,Nbins,-1,1)
reldiffty_vvuu=TH2F("reldiffty_vvuu", "", Nbins,5000,50000,Nbins,-1,1)


#th2f p vs diff_vvuu
for index in range(0,len(p_list)):
    difftx_vvuu.Fill(p_list[index],difftx_vv_uu_list[index])
    diffty_vvuu.Fill(p_list[index],diffty_vv_uu_list[index])
    if(txVeloVelo_list[index] != 0):
        reldifftx_vvuu.Fill(p_list[index],difftx_vv_uu_list[index]/txVeloVelo_list[index])
    if(tyVeloVelo_list[index] != 0):
        reldiffty_vvuu.Fill(p_list[index],diffty_vv_uu_list[index]/tyVeloVelo_list[index])
    

    
Nbins = 60
tx_vv=TH1F("tx_vv", "", Nbins,-0.25,0.25)
ty_vv=TH1F("ty_vv", "", Nbins,-0.25,0.25)

#th1f slope vv
for index in range(0,len(p_list)):
    tx_vv.Fill(txVeloVelo_list[index])
    ty_vv.Fill(tyVeloVelo_list[index])
    


3582
1367


Warning in <TROOT::Append>: Replacing existing TH1: difftx_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: diffty_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: reldifftx_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: reldiffty_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: tx_vv (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ty_vv (Potential memory leak).


In [98]:
c4=TCanvas("c4","",1300,1800)
c4.Divide(2,3)

c4.cd(1)
difftx_vvuu.Draw()
difftx_vvuu.GetXaxis().SetTitle("p")
difftx_vvuu.GetYaxis().SetTitle("diffx_vvuu")
difftx_vvuu.SetMarkerSize(0.5)

c4.cd(2)
diffty_vvuu.Draw()
diffty_vvuu.GetXaxis().SetTitle("p")
diffty_vvuu.GetYaxis().SetTitle("diffy_vvuu")
diffty_vvuu.SetMarkerSize(0.5)

c4.cd(3)
reldifftx_vvuu.Draw()
reldifftx_vvuu.GetXaxis().SetTitle("p")
reldifftx_vvuu.GetYaxis().SetTitle("difftx_vvuu/txvv")
reldifftx_vvuu.SetMarkerSize(0.5)

c4.cd(4)
reldiffty_vvuu.Draw()
reldiffty_vvuu.GetXaxis().SetTitle("p")
reldiffty_vvuu.GetYaxis().SetTitle("diffty_vvuu/tyvv")
reldiffty_vvuu.SetMarkerSize(0.5)


c4.cd(5)
tx_vv.Draw()
tx_vv.GetXaxis().SetTitle("tx_vv")
tx_vv.GetYaxis().SetTitle("Events")

c4.cd(6)
ty_vv.Draw()
ty_vv.GetXaxis().SetTitle("tx_vv")
ty_vv.GetYaxis().SetTitle("Events")


c4.SaveAs("slopes_vvuu.png")




Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Info in <TCanvas::Print>: png file slopes_vvuu.png has been created
